<a href="https://colab.research.google.com/github/vasiliyeskin/MachineLearningExperiences/blob/master/transfer_learning/cats_and_dogs_vgg16_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Подготовка данных для распознавания котов и собак на изображениях**
Источник данных - соревнования Kaggle Dogs vs. Cats.

Пример подготовки данных для обучения нейронной сети на Keras. Данные разбиваются на три каталога:

train (данные для обучения)
val (данные для проверки)
test (данные для тестирования)
В каждом каталоге создаются по два подкаталога, в соответсвии с названиями классов: cats и dogs.

Изображения переписваются из исходного каталога в новую структуру. По-умолчанию для обучения используется 70% изображений, для проверки - 15%, для тестрования также 15%.

Перед запуском необходимо скачать файл train.zip с набором изображений кошек и собак с сайта соревнования Kaggle Dogs vs. Cats и распаковать его.

In [50]:
import shutil
import os

In [ ]:
!pip install tensorflow-datasets

In [52]:
import tensorflow_datasets as tfds

In [55]:
train = tfds.load('cats_vs_dogs', split='train')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete9CJYWQ/cats_vs_dogs-train.tfrecord



Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [62]:
# Каталог с набором данных
data_dir = '/root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete9CJYWQ/cats_vs_dogs-train.tfrecord'
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = 12500

Функция создания каталога с двумя подкаталогами по названию классов: cats и dogs

In [63]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "cats"))
    os.makedirs(os.path.join(dir_name, "dogs"))


Создание структуры каталогов для обучающего, проверочного и тестового набора данных

In [64]:
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

Функция копирования изображений в заданный каталог. Изображения котов и собак копируются в отдельные подкаталоги

In [65]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "cat." + str(i) + ".jpg"), 
                    os.path.join(dest_dir, "cats"))
        shutil.copy2(os.path.join(source_dir, "dog." + str(i) + ".jpg"), 
                   os.path.join(dest_dir, "dogs"))

Расчет индексов наборов данных для обучения, приверки и тестирования

In [66]:
start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
start_test_data_idx = int(nb_images * (1 - test_data_portion))
print(start_val_data_idx)
print(start_test_data_idx)

8750
10625



Копирование изображений

In [67]:
copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir)
copy_images(start_test_data_idx, nb_images, data_dir, test_dir)

FileNotFoundError: ignored

# Распознавание собак и кошек на изображениях с помощью предварительно обученной нейронной сети VGG16

Источник данных - соревнования Kaggle Dogs vs. Cats.

Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования. Можно использовать пример в ноутбуке data_preparation

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam

In [12]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750


**Загружаем предварительно обученную нейронную сеть**

In [22]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

"Замораживаем" веса предварительно обученной нейронной сети VGG16

In [23]:
vgg16_net.trainable = False

In [ ]:
vgg16_net.summary()

**Создаем составную нейронную сеть на основе VGG16**

In [25]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()


Компилируем составную нейронную сеть

In [27]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

**Создаем генератор изображений**
Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [28]:
datagen = ImageDataGenerator(rescale=1. / 255)

Генератор данных для обучения на основе изображений из каталога

In [29]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

FileNotFoundError: ignored

Генератор данных для проверки на основе изображений из каталога

In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Генератор данных для тестирования на основе изображений из каталога

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


# **Обучаем модель с использованием генераторов**
train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

# **Оцениваем качество работы сети с помощью генератора**

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [ ]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))